# Chatbot with Conversational History using LCEL

In [3]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "gemma2-9b-It", groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001DE269C2540>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001DE26B04E90>, model_name='gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content = "Hii, I am Yash. I am a Data Sceintist")])

AIMessage(content="Hello Yash! 👋\n\nIt's great to meet you. Being a Data Scientist is awesome! What kind of projects are you working on these days?  \n\nDo you have any questions for me about data science, or anything else? I'm here to help! 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 22, 'total_tokens': 83, 'completion_time': 0.110909091, 'prompt_time': 0.00015399, 'queue_time': 0.013471419, 'total_time': 0.111063081}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e85f87c6-a613-4024-8ab9-368c4613e08b-0', usage_metadata={'input_tokens': 22, 'output_tokens': 61, 'total_tokens': 83})

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content = "Hii, I am Yash. I am a Data Sceintist"),
        AIMessage(content = "Hello Yash! 👋\n\nIt's great to meet you. Being a Data Scientist is awesome! What kind of projects are you working on these days?  \n\nDo you have any questions for me about data science, or anything else? I'm here to help! 😊 \n\n"),
        HumanMessage(content = "What is my name and what do I do?"),
    ]
)

AIMessage(content="You told me your name is Yash, and that you are a Data Scientist!  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm curious to learn more. 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 100, 'total_tokens': 149, 'completion_time': 0.089090909, 'prompt_time': 0.002506402, 'queue_time': 0.012826656, 'total_time': 0.091597311}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-23b259d7-243a-49f6-8ad2-5bc31a3977d9-0', usage_metadata={'input_tokens': 100, 'output_tokens': 49, 'total_tokens': 149})

It seems that our model is remembering out history.

### **Associating Conversation History with SessionId**

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

config = {
    "configurable":{
        "session_id": "chat1"
    }
}

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content= "Hi my name is Yash, I am a Data Scientist")],
    config = config
)
response.content

"It's nice to meet you, Yash!  \n\nWhat kind of data science work do you do?  Are you working on any interesting projects right now? 😊 \n\nI'm always eager to learn more about how people are using data science to make a difference.  \n\n"

In [10]:
with_message_history.invoke(
    [
        HumanMessage(content= "What is my name")
    ],
    config = config
)

AIMessage(content='Your name is Yash. 😊  \n\nI remember that you told me at the beginning of our conversation.  \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 173, 'total_tokens': 199, 'completion_time': 0.047272727, 'prompt_time': 0.005363543, 'queue_time': 0.008265204, 'total_time': 0.05263627}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f330d6ef-e88d-4ba5-b3d6-9b3b5fc63b12-0', usage_metadata={'input_tokens': 173, 'output_tokens': 26, 'total_tokens': 199})

In [12]:
config2 = {
    "configurable":{
        "session_id": "chat2"
    }
}

with_message_history.invoke(
    [
        HumanMessage(content= "Hii What is my name?")
    ],
    config = config2
)

AIMessage(content="As an AI, I have no memory of past conversations and don't know your name. If you'd like to tell me, I'd be happy to use it! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 15, 'total_tokens': 58, 'completion_time': 0.078181818, 'prompt_time': 8.394e-05, 'queue_time': 0.01315726, 'total_time': 0.078265758}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fbf131c1-d6c1-40b6-ab9c-51d9ae70a622-0', usage_metadata={'input_tokens': 15, 'output_tokens': 43, 'total_tokens': 58})

As you can see, if we change config, it will not have info about other sessions

### **Prompt Templates**

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all the questions to the best of your ability"),
    MessagesPlaceholder(variable_name= "messages")
])

chain = prompt | model

In [14]:
chain.invoke({
    "messages":[HumanMessage(content = "Hii, I am Yash. I am a Data Sceintist")]
})

AIMessage(content="Hi Yash, it's nice to meet you!\n\nI understand you're a Data Scientist. That's awesome!  \n\nWhat can I help you with today?  Do you have any questions about data science, need help with a specific problem, or just want to chat about the field?  \n\nI'm here to assist in any way I can. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 38, 'total_tokens': 120, 'completion_time': 0.149090909, 'prompt_time': 0.000588848, 'queue_time': 0.013633351, 'total_time': 0.149679757}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2467451b-9cd2-4679-9e7f-ad941f8a2a51-0', usage_metadata={'input_tokens': 38, 'output_tokens': 82, 'total_tokens': 120})

In [15]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
response = with_message_history.invoke(
    [HumanMessage(content= "Hi my name is Yash, I am a Data Scientist")],
    config = config
)

response

AIMessage(content="Hi Yash! It's nice to meet you.  \n\nBeing a Data Scientist is awesome!  What kind of projects are you working on these days? Do you have a favorite area of data science? \n\nI'm always eager to learn more about how people are using data to solve problems and make the world a better place. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 233, 'total_tokens': 308, 'completion_time': 0.136363636, 'prompt_time': 0.008397194, 'queue_time': 0.005367754000000001, 'total_time': 0.14476083}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4ece1f8e-1641-4ec4-a9e6-2b7fdbc6d403-0', usage_metadata={'input_tokens': 233, 'output_tokens': 75, 'total_tokens': 308})

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content= "What is my name")],
    config = config
)

response.content

'Your name is Yash.  \n\nI remember! 😊  \n\nIs there anything else I can help you with?'

In [23]:
# adding more complexity 
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant. Answer all the questions to the best of your ability in {language}",

    ),
    MessagesPlaceholder(variable_name = "messages")
])

chain = prompt | model 

In [24]:
response = chain.invoke({
    "messages": [HumanMessage(content = "Tell me about Generative Ai ")],
    "language": "Marathi"
})

response.content

'## जेनरेटिव्ह आय.ए.आय. (Generative AI)\n\nजेनरेटिव्ह आय.ए.आय. हे एक प्रकारचे आर्टिफिशियल इंटेलिजेन्समध्ये लक्ष केंद्रित होते ज्यामध्ये माहिती पैकी नवीन माहिती तयार करण्याची क्षमता असते.\n\n**जेनरेटिव्ह आय.ए.आय. काय करते?**\n\nजेनरेटिव्ह आय.ए.आय. डेटा सेट्सचे विश्लेषण करून आणि त्यांमधून शिकून नवीन डेटा तयार करतो. हे डेटा विविध स्वरूपात असू शकते, जसे की:\n\n* **टेक्स्ट:** कविता, कादंबऱ्या, लेख, कथा इ.\n* **इमेज:** चित्र, फोटो, कलाकृती इ.\n* **ऑडिओ:** संगीत, भाषण, आवाज इ.\n* **व्हिडिओ:** अॅनिमेशन, सिनेमा, व्हिडिओ गेम इ.\n\n**जेनरेटिव्ह आय.ए.आय. कशा प्रकारच्या कार्ये करू शकते?**\n\n* **नवीन कथा लिहिणे:** तुम्हाला कल्पना हवी असेल तर, जेनरेटिव्ह आय.ए.आय. तुम्हाला एका कथेची सुरुवात करण्यास मदत करू शकते, किंवा पूर्ण कथा लिहिण्यास मदत करू शकते.\n* **चित्र बनवणे:** तुमच्या मनात एक चित्र आहे पण तुम्ही तो स्वतः बनवू शकत नाही, तर जेनरेटिव्ह आय.ए.आय. तुम्हाला त्या चित्राचे डिजिटल रूप प्रदान करू शकते.\n* **संगीत तयार करणे:** जर तुम्हाला संगीताची आवड असेल तर, जेनरेटिव्ह आय.ए.आय. तुम्हाला नवीन संगीत र

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [25]:
with_messgage_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key = "messages"
)

In [26]:
config = {
    "configurable": {
        "session_id": "Chat3"
    }
}

In [29]:
response = with_messgage_history.invoke({
    'messages': [HumanMessage(content = "Tell me about Generative Ai ")],
    'language' : "Spanish",
},
config = config
)

response.content

'## ¡Con gusto te hablo sobre la IA Generativa!\n\nLa Inteligencia Artificial Generativa (IA Generativa) es un tipo emocionante de aprendizaje automático que se enfoca en crear nuevos contenidos. Imagina una máquina que puede escribir historias, componer música, generar imágenes o incluso diseñar código, ¡eso es lo que hace la IA Generativa!\n\n**¿Cómo funciona?**\n\nLa IA Generativa aprende a partir de grandes conjuntos de datos existentes. Analiza patrones y estructuras en ese data para luego poder generar nuevos ejemplos que se ajusten a esos mismos patrones. \n\n**Algunos ejemplos de aplicaciones:**\n\n* **Creación de texto:**\n\nChatbots, redacción de artículos, traducción automática, generación de poesía y guiones.\n* **Generación de imágenes:**\n\nCreación de imágenes realistas o abstractas a partir de descripciones textuales, diseño de logos y personajes.\n* **Composición musical:**\n\nGeneración de melodías, armonías y ritmos nuevos.\n* **Diseño:**\n\nGeneración de diseños par

In [30]:
response = with_messgage_history.invoke({
    'messages': [HumanMessage(content = "What did I ask about? ")],
    'language' : "English",
},
config = config
)

response.content

"You asked me to tell you about Generative AI.  \n\nI explained what it is, how it works, some examples of its applications, and some of the key technologies behind it. \n\nIs there anything else you'd like to know about Generative AI?  \n\n"

### Managing the Conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [31]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens = 54,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

d:\GEN AI\Langchain\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\GEN AI\Langchain\myenv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [33]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content= "Hi my name is Yash, I am a Data Scientist")],
        "language":"Hindi"
    }
)

In [34]:
response.content

'Namaste Yash!  मुझे खुशी है आपसे मिलने को।  आप एक Data Scientist हैं, यह बहुत ही रोचक है! \n\nक्या मैं आपकी कोई मदद कर सकता हूँ?  \n'

In [35]:
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

'You said you like **vanilla** ice cream! 🍦  \n\nIs there something else I can help you with?\n'

In [36]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked: "what\'s 2 + 2" \n\nIs there anything else I can help you with?\n'

In [38]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I don't have access to any information about you, including your name.  If you'd like to tell me your name, I'd be happy to know! 😊  \n"

It forgot as we are using Trimmer 